In [41]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("/Users/guilhermesilva/UFMG/Mestrado/Thesis/tinyc")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# `src.abstract_syntax_tree`

In [42]:
from src.abstract_syntax_tree import Node, AbstractSyntaxTree

## `Node`

In [43]:
node_with_set_value = Node(id=1, kind="TEST", value=1)
print(node_with_set_value)

ID: 1, Kind: TEST, Value: 1


In [44]:
node_with_default_value = Node(id=1, kind="TEST")
print(node_with_default_value)

ID: 1, Kind: TEST, Value: -1


## `AbstractSyntaxTree`

In [45]:
tree = AbstractSyntaxTree()
print(tree.root)

ID: 0, Kind: PROG, Value: -1


# `src.syntax_parser`

In [46]:
from src.syntax_parser import SyntaxParser

## `SyntaxParser`

In [47]:
parser = SyntaxParser()
vars(parser)

{'_global_id_manager': 1}

### `term`

In [48]:
test_id_term = parser.term("ID", 1)
print(test_id_term)

ID: 1, Kind: VAR, Value: 1


In [49]:
# This should raise Exceptions, as the call is malformed.

try:
    test_failed_term = parser.term(
        symbol="OTHER",
        id=1,
        lhs=Node(-1, "TEST"),
        rhs=Node(-2, "TEST")
    )
    print(test_failed_term)
except UnboundLocalError as e:
    print(e)
    print("Test case 1: all parameters were set")

try:
    test_failed_term = parser.term(
        symbol=["OTHER"],
        id=1
    )
    print(test_failed_term)
except UnboundLocalError as e:
    print(e)
    print("Test case 2: `term` case, but symbol is a list")

try:
    test_failed_term = parser.term(
        symbol="OTHER",
        lhs=Node(-1, "TEST"),
        rhs=Node(-2, "TEST")
    )
    print(test_failed_term)
except UnboundLocalError as e:
    print(e)
    print("Test case 3: `parenthesis_expression` case, but symbol is a str")

ERROR: term() got an unexpected keyword argument 'id'
Could not run decorated function. The ID counter was not affected.
local variable 'result' referenced before assignment
Test case 1: all parameters were set
ERROR: term() got an unexpected keyword argument 'id'
Could not run decorated function. The ID counter was not affected.
local variable 'result' referenced before assignment
Test case 2: `term` case, but symbol is a list
ERROR: too many values to unpack (expected 3)
Could not run decorated function. The ID counter was not affected.
local variable 'result' referenced before assignment
Test case 3: `parenthesis_expression` case, but symbol is a str


In [50]:
test_int_term = parser.term("INT", 1)
print(test_int_term)

ID: 2, Kind: CST, Value: 1


In [51]:
test_parenthesis_term = parser.term(
    symbol=["LPAR", "ID", "RPAR"],
    lhs=Node(-1, "TEST_LHS"),
    rhs=Node(-2, "TEST_RHS")
)
print(test_parenthesis_term)

for child in test_parenthesis_term.children:
    print(child)

ID: 3, Kind: SET, Value: -1
ID: -1, Kind: TEST_LHS, Value: -1, Parent ID: 3
ID: -2, Kind: TEST_RHS, Value: -1, Parent ID: 3


### `parenthesis_expression`

In [52]:
lhs = Node(id=-1, kind="INT", value=1)
rhs = Node(id=-2, kind="INT", value=2)

In [53]:
# Generates a SET node

test_parenthesis_expression_set = parser.parenthesis_expression(
    symbol=["LPAR", "ID", "RPAR"],
    lhs=lhs,
    rhs=rhs
)
print(test_parenthesis_expression_set)

for child in test_parenthesis_expression_set.children:
    print(child)

ID: 5, Kind: SET, Value: -1
ID: -1, Kind: INT, Value: 1, Parent ID: 5
ID: -2, Kind: INT, Value: 2, Parent ID: 5


In [54]:
# Generates a LT node

test_parenthesis_expression_lt = parser.parenthesis_expression(
    symbol=["LPAR", "OTHER", "RPAR"],
    lhs=lhs,
    rhs=rhs
)
print(test_parenthesis_expression_lt)

for child in test_parenthesis_expression_lt.children:
    print(child)

ID: 6, Kind: LT, Value: -1
ID: -1, Kind: INT, Value: 1, Parent ID: 6
ID: -2, Kind: INT, Value: 2, Parent ID: 6


In [55]:
# Raises Exceptions, as the parenthesis are not correctly placed.

try:
    test_parenthesis_expression_placement = parser.parenthesis_expression(
        symbol=["RPAR", "OTHER", "LPAR"],
        lhs=lhs,
        rhs=rhs
    )
    print(test_parenthesis_expression_placement)
except Exception as e:
    print(e)
    print("Test case 1: the RPAR is before LPAR")

try:
    test_parenthesis_expression_missing = parser.parenthesis_expression(
        symbol=["RPAR", "OTHER"],
        lhs=lhs,
        rhs=rhs
    )
    print(test_parenthesis_expression_missing)
except Exception as e:
    print(e)
    print("Test case 2: missing LPAR")

try:
    test_parenthesis_expression_missing = parser.parenthesis_expression(
        symbol=["LPAR", "OTHER"],
        lhs=lhs,
        rhs=rhs
    )
    print(test_parenthesis_expression_missing)
except Exception as e:
    print(e)
    print("Test case 2: missing RPAR")

Missing or misplaced left parenthesis symbol (LPAR)
Test case 1: the RPAR is before LPAR
not enough values to unpack (expected 3, got 2)
Test case 2: missing LPAR
not enough values to unpack (expected 3, got 2)
Test case 2: missing RPAR


### `expression`

In [56]:
lhs = Node(id=-1, kind="INT", value=1)
rhs = Node(id=-2, kind="INT", value=2)

In [57]:
test_expression_lt = parser.expression(
    symbol="OTHER",
    lhs=lhs,
    rhs=rhs
)
print(test_expression_lt)

for child in test_expression_lt.children:
    print(child)

ID: 8, Kind: LT, Value: -1
ID: -1, Kind: INT, Value: 1, Parent ID: 8
ID: -2, Kind: INT, Value: 2, Parent ID: 8


In [58]:
test_expression_set = parser.expression(
    symbol="ID",
    lhs=lhs,
    rhs=rhs
)
print(test_expression_set)

for child in test_expression_set.children:
    print(child)

ID: 10, Kind: SET, Value: -1
ID: -1, Kind: INT, Value: 1, Parent ID: 10
ID: -2, Kind: INT, Value: 2, Parent ID: 10


### `sum`

In [59]:
lhs = Node(id=-1, kind="INT", value=1)
rhs = Node(id=-2, kind="INT", value=2)

In [60]:
test_sum_plus = parser.sum(symbol="PLUS", lhs=lhs, rhs=rhs)
print(test_sum_plus)

for child in test_sum_plus.children:
    print(child)
    print(child.parent)

ID: 11, Kind: ADD, Value: -1
ID: -1, Kind: INT, Value: 1, Parent ID: 11
ID: 11, Kind: ADD, Value: -1
ID: -2, Kind: INT, Value: 2, Parent ID: 11
ID: 11, Kind: ADD, Value: -1


In [61]:
test_sum_minus = parser.sum(symbol="MINUS", lhs=lhs, rhs=rhs)
print(test_sum_minus)

for child in test_sum_minus.children:
    print(child)
    print(child.parent)

ID: 12, Kind: SUB, Value: -1
ID: -1, Kind: INT, Value: 1, Parent ID: 12
ID: 12, Kind: SUB, Value: -1
ID: -2, Kind: INT, Value: 2, Parent ID: 12
ID: 12, Kind: SUB, Value: -1


### `comparison`

In [62]:
lhs = Node(id=-1, kind="INT", value=1)
rhs = Node(id=-2, kind="INT", value=2)

In [63]:
test_comparison = parser.comparison(lhs=lhs, rhs=rhs)
print(test_comparison)

for child in test_comparison.children:
    print(child)
    print(child.parent)

ID: 13, Kind: LT, Value: -1
ID: -1, Kind: INT, Value: 1, Parent ID: 13
ID: 13, Kind: LT, Value: -1
ID: -2, Kind: INT, Value: 2, Parent ID: 13
ID: 13, Kind: LT, Value: -1


### `if_statement`

In [64]:
_parenthesis_expression = Node(id=1, kind="PAR")
_statement = Node(id=2, kind="STTMNT")

In [65]:
if_statement = parser.if_statement(_parenthesis_expression, _statement)
print(if_statement)

for child in if_statement.children:
    print(child)

ID: 14, Kind: IF, Value: -1
ID: 1, Kind: PAR, Value: -1, Parent ID: 14
ID: 2, Kind: STTMNT, Value: -1, Parent ID: 14


### `if_else_statement`

In [66]:
_parenthesis_expression = Node(id=1, kind="PAR")
_statement_if = Node(id=2, kind="STTMNT")
_statement_else = Node(id=3, kind="STTMNT")

In [67]:
if_else_statement = parser.if_else_statement(_parenthesis_expression, _statement_if, _statement_else)
print(if_else_statement)

for child in if_else_statement.children:
    print(child)

ID: 15, Kind: IFELSE, Value: -1
ID: 1, Kind: PAR, Value: -1, Parent ID: 15
ID: 2, Kind: STTMNT, Value: -1, Parent ID: 15
ID: 3, Kind: STTMNT, Value: -1, Parent ID: 15


### `while_statement`

In [69]:
_parenthesis_expression = Node(id=1, kind="PAR")
_statement = Node(id=2, kind="STTMNT")

In [70]:
while_statement = parser.while_statement(_parenthesis_expression, _statement)
print(while_statement)

for child in while_statement.children:
    print(child)

ID: 16, Kind: WHILE, Value: -1
ID: 1, Kind: PAR, Value: -1, Parent ID: 16
ID: 2, Kind: STTMNT, Value: -1, Parent ID: 16


### `do_while_statement`

In [71]:
_parenthesis_expression = Node(id=1, kind="PAR")
_statement = Node(id=2, kind="STTMNT")

In [72]:
do_while_statement = parser.do_while_statement(_parenthesis_expression, _statement)
print(do_while_statement)

for child in do_while_statement.children:
    print(child)

ID: 17, Kind: DOWHILE, Value: -1
ID: 1, Kind: PAR, Value: -1, Parent ID: 17
ID: 2, Kind: STTMNT, Value: -1, Parent ID: 17


### `program`

In [73]:
_statement = Node(id=1, kind="STTMNT")

In [74]:
program = parser.program(_statement)
print(program)

for child in program.children:
    print(child)

ID: 18, Kind: PROG, Value: -1
ID: 1, Kind: STTMNT, Value: -1, Parent ID: 18
